Copyright (C) Egon Kidmose 2015-2017

This file is part of lstm-rnn-correlation.

lstm-rnn-correlation is free software: you can redistribute it and/or
modify it under the terms of the GNU Lesser General Public License as
published by the Free Software Foundation, either version 3 of the
License, or (at your option) any later version.

lstm-rnn-correlation is distributed in the hope that it will be
useful, but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU
Lesser General Public License for more details.

You should have received a copy of the GNU Lesser General Public
License along with lstm-rnn-correlation. If not, see
<http://www.gnu.org/licenses/>.


# Alert correlation with a Long Short-Term Memoroy (LSTM) Recurrent Neural Network(RNN) and cosine similarity

**Author:** Egon Kidmose, egk@es.aau.dk

In network security a common task is to detect network intrusions and for this purpose an Intrusion Detections System (IDS) can be used to raise alerts on suspicious network traffic.
Snort, Suricata and Bro are examples of free and open source IDSs (Commercial products also exist).
The alerts generally provides low level information such as recognition of strings that are known to be part of security exploits or anomalous connection rates for a host.
By grouping alerts that are correlated into higher level events, false positives might be suppressed and attack scenarios becomes easier to recognise. 
This is a take on how to correlate IDS alerts to determine which belong in the same group.

Alerts can be represented as log lines with various information such as time stamp, IP adresses, protocol information and a description of what triggered the alert. 
It is assumed that such a log lines hold the information needed to determine if two alerts are correlated or not.

The input to the neural network will be two alerts and the output will indicate if they are correlated or not. 
In further detail the inputs is two strings of ASCII characters of variable length. 
For the output a Cosine Similarity layer is implemented and used to produce an output in the range [-1,1], with -1 meaning opposite, 0 meaning orthogonal and 1 meaning the same. 

For the hidden layers only a single layers of Long Short-Term Memory (LSTM) cells is used.
It is an option to experiment with adding more.
Being reccurrent, such a layer handles variable length input well. 

While it turned out to be to challenging to implement, the initial idea was to let the two inputs pass through LSTM layers with identical weights.
The intent was to have them act as transformations into a space where cosine similarity could be used to measure similarity of the alerts.
However I have not succeded at tying the weights together.
As an alternative this might be achieved by using all training pairs in both original and swapped order.
The intuition is that this leads to two identical layers, but intuition also suggest that this is highly ineffective.

                      Output
                        |
    Cosine similarity   #
                       / \
        LSTM layers   #   #
                      |   |
        "alert number 1"  |
            "alert number 2"


Reference: Huang, Po-Sen, et al. "Learning deep structured semantic models for web search using clickthrough data." Proceedings of the 22nd ACM international conference on Conference on information & knowledge management. ACM, 2013.


In [ ]:
from __future__ import print_function
from __future__ import division

import sys
import os
import time
import glob

import numpy as np
import theano
import theano.tensor as T

import lasagne
from lasagne.layers import *
from lasagne.nonlinearities import *
from lasagne.updates import *
from lasagne.objectives import *

import lstm_rnn_tied_weights
from lstm_rnn_tied_weights import CosineSimilarityLayer

In [ ]:
# Data for unit testing
X_unit = ['abcdef', 'abcdef', 'qwerty']
X_unit = [[ord(c) for c in w] for w in X_unit]
X_unit = np.array(X_unit, dtype='int8')
print(X_unit)
n_alerts_unit, l_alerts_unit = X_unit.shape
mask_unit = np.ones(X_unit.shape, dtype='int8')
print(mask_unit)

In [ ]:
# Dimensions
n_alerts = None
l_alerts = None
n_alphabet = 2**7 # All ASCII chars
num_units = 10

In [ ]:
# Symbolic variables
input_var, input_var2 = T.imatrices('inputs', 'inputs2')
mask_var, mask_var2 = T.matrices('masks', 'masks2')
target_var = T.dvector('targets')

In [ ]:
# First line
l_in = InputLayer(shape=(n_alerts, l_alerts), input_var=input_var, name='INPUT-LAYER')
l_emb = EmbeddingLayer(l_in, n_alphabet, n_alphabet, 
                         W=np.eye(n_alphabet),
                         name='EMBEDDING-LAYER')
l_emb.params[l_emb.W].remove('trainable') # Fix weight
l_mask = InputLayer(shape=(n_alerts, l_alerts), input_var=mask_var, name='MASK-INPUT-LAYER')
l_lstm = LSTMLayer(l_emb, num_units=num_units, name='LSTM-LAYER', mask_input=l_mask)
l_slice = SliceLayer(l_lstm, indices=-1, axis=1, name="SLICE-LAYER") # Only last timestep
net = l_slice

In [ ]:
# Test first line

# Test InputLayer
pred_unit = get_output(l_in, inputs={l_in: input_var}).eval(
    {input_var: X_unit})
assert (pred_unit == X_unit).all(), "Unexpected output"
# Test EmbeddingLayer
pred_unit = get_output(l_emb, inputs={l_in: input_var}).eval(
    {input_var: X_unit})
assert (np.argmax(pred_unit, axis=2) == X_unit).all()
assert np.all(pred_unit.shape == (n_alerts_unit, l_alerts_unit, n_alphabet ))
# Test LSTMLayer
pred_unit = get_output(
    l_lstm,
    inputs={l_in: input_var, l_mask: mask_var}
).eval({input_var: X_unit, mask_var: mask_unit})
assert pred_unit.shape == (n_alerts_unit, l_alerts_unit, num_units), "Unexpected dimensions"
pred_unit = get_output(
    l_lstm, 
    inputs={l_in: input_var, l_mask: mask_var}
).eval({input_var: [[1],[1]], mask_var: [[1],[1]]})
assert np.all(pred_unit[0] == pred_unit[1]), "Repeated alerts must produce the same"
pred_unit = get_output(
    l_lstm,
    inputs={l_in: input_var, l_mask: mask_var}
).eval({input_var: [[1,1],[1,1]], mask_var: [[1,1],[1,1]]})
assert np.all(pred_unit[0] == pred_unit[1]), "Repeated alerts must produce the same"
pred_unit = get_output(
    l_lstm,
    inputs={l_in: input_var, l_mask: mask_var}
).eval({input_var: [[1,1],[0,1]], mask_var: [[1,1],[1,1]]})
assert np.all(pred_unit[0] != pred_unit[1]), "Earlier must affect laters"
pred_unit = get_output(
    l_lstm,
    inputs={l_in: input_var, l_mask: mask_var}
).eval({input_var: [[1,0],[1,1]], mask_var: [[1,1],[1,1]]})
assert np.all(pred_unit[0,0] == pred_unit[1,0]), "Later must not affect earlier"
assert np.all(pred_unit[0,1] != pred_unit[1,1]), "Current must make a difference"
# Test SliceLayer
pred_unit = get_output(
    l_slice,
    inputs={l_in: input_var, l_mask: mask_var}
).eval({input_var: X_unit, mask_var: mask_unit})
assert pred_unit.shape == (n_alerts_unit, num_units), "Unexpected shape"
pred_unit_lstm = get_output(
    l_lstm,
    inputs={l_in: input_var, l_mask: mask_var}
).eval({input_var: X_unit, mask_var: mask_unit})
assert np.all(pred_unit_lstm[:, -1, :] == pred_unit), "Unexpected result of slicing"

print('OK')

In [ ]:
# Second line as a copy with shared weights
l_in2 = InputLayer(shape=l_in.shape, input_var=input_var2, name=l_in.name+'2')
l_mask2 = InputLayer(shape=l_mask.shape, input_var=mask_var2, name=l_mask.name+'2')
net2 = lstm_rnn_tied_weights.clone(net, l_in2, l_mask2)



In [ ]:
# Merge lines
l_cos = CosineSimilarityLayer(net, net2, name="COSINE-SIMILARITY-LAYER")
l_sig = NonlinearityLayer(l_cos, nonlinearity=sigmoid, name="SIGMOID-LAYER")
cos_net = l_sig

In [ ]:
# Test
pred_unit = get_output(l_cos, inputs={
        l_in: input_var,
        l_mask: mask_var,
        l_in2: input_var2,
        l_mask2: mask_var2,
    }).eval({
        input_var: X_unit,
        mask_var: mask_unit,
        input_var2: X_unit,
        mask_var2: mask_unit,
    })

assert pred_unit.shape == (n_alerts_unit,)

print('OK')

In [ ]:
# Training Procedure
t = time.time()
prediction = get_output(cos_net)
loss = binary_crossentropy(prediction, target_var)
loss = loss.mean()
params = get_all_params(cos_net, trainable=True)
updates = sgd(loss, params, learning_rate=0.1)

# Testing Procedure
test_prediction = get_output(cos_net, deterministic=True)
test_loss = binary_crossentropy(test_prediction, target_var)
test_loss = test_loss.mean()
test_acc = T.mean(T.eq(test_prediction > 0.5, target_var),
                  dtype=theano.config.floatX)

train_fn = theano.function([input_var, input_var2, mask_var, mask_var2, target_var], loss, updates=updates)
val_fn = theano.function([input_var, input_var2, mask_var, mask_var2, target_var], [test_loss, test_acc])
print("Spent {}s compilling.".format(time.time()-t))

In [ ]:
# Data preparation / loading functions
def load_data(
    file_names,
    max_alerts=None,
):
    start_time = time.time()
    print("Loading {} files:".format(len(file_names)))
    
    alerts = list()
    incidents = list()
    for i, fn in enumerate(file_names):
        i += 1
        print(' - {}/{} {}'.format(i, len(file_names), fn))
        with open(fn, 'r') as f:
            for l in f.readlines():
                alerts.append(l)
                incidents.append(i)

    lens = list(map(len, alerts))
    alert_matrix = np.zeros((len(alerts),max(lens)), dtype='int8')
    mask_matrix = np.zeros_like(alert_matrix)
    for i, alert in enumerate(alerts):
        mask_matrix[i, :lens[i]] = 1
        for j, c in enumerate(alert):
            alert_matrix[i,j] = ord(c)

    print("Completed loading {} alerts in {}s".format(len(alert_matrix), time.time()-start_time))
    
    if max_alerts is not None:
        print("Capping data to {} alerts".format(max_alerts))
        return alert_matrix[:max_alerts], mask_matrix[:max_alerts], incidents[:max_alerts]
    
    return alert_matrix, mask_matrix, incidents

def split_data(
    alerts, 
    masks, 
    incidents,
    train_weight=60,
    val_weight=20,
    test_weight=20,
):
    """Split data into training, validation and test sets"""
    assert len(alerts) == len(masks)
    assert len(alerts) == len(incidents)
    assert alerts.shape == masks.shape
    n = len(alerts)
    
    weights = np.array([train_weight, val_weight, test_weight])
    weights = (weights/sum(weights)*n).astype(int)
    idxs = list(np.cumsum(weights))
    idxs = [0] + idxs
    
    print("Splitting {} samples into: ".format(len(alerts)) + str(weights))
    for start, end in zip (idxs[:-1], idxs[1:]):
        yield alerts[start:end], masks[start:end], incidents[start:end]

def cross_join(
    cut,
    max_alerts=None,
):
    """Cross join list of alerts with self and track if incident is the same."""
    alerts, masks, incidents = cut
    assert len(alerts) == len(masks)
    assert len(alerts) == len(incidents)
    assert alerts.shape == masks.shape
    
    # Shuffle to sample across all alerts in a predictive fashion
    np.random.seed(1131662768)
    i_idxs = np.arange(len(alerts))
    np.random.shuffle(i_idxs)
    j_idxs = np.arange(len(alerts))
    np.random.shuffle(j_idxs)
    
    if max_alerts is not None:
        print("Capping to {} alert pairs.".format(max_alerts))
        i_idxs = i_idxs[:max_alerts]
        j_idxs = j_idxs[:max_alerts]
    
    for i in i_idxs:
        for j in j_idxs:
            yield (
                alerts[i],
                alerts[j],
                masks[i],
                masks[j],
                incidents[i] == incidents[j],
            )

def iterate_minibatches(samples, batch_size):
    # Sneek peak at first sample to learn alert length
    sample = next(samples)
    alert1, alert2, mask1, mask2, correlation = sample
    inputs1 = np.empty((batch_size, len(sample[0])), dtype=alert1.dtype)
    inputs2 = np.empty_like(inputs1)
    masks1 = np.empty_like(inputs1)
    masks2 = np.empty_like(inputs1)
    targets = np.empty((batch_size), dtype=bool)
    i = 0 # index for the arrays

    # Remember to use first samples
    inputs1[i], inputs2[i], masks1[i], masks2[i], targets[i] = sample
    i += 1
    
    # Use remaining samples to build and yield batches
    batches_produced = 0
    samples_processed = 0
    for sample in samples:
        inputs1[i], inputs2[i], masks1[i], masks2[i], targets[i] = sample
        i += 1
        if i == batch_size: 
            batches_produced += 1
            samples_processed += i
            i = 0
            yield inputs1, inputs2, masks1, masks2, targets
    samples_processed += i
    assert batches_produced > 0, "{} samples is not enough to produce a batch({} samples)".format(
        samples_processed, batch_size)
    batches_expected = samples_processed // batch_size
    assert batches_expected == batches_produced, "Expected {} but produced {} batches". format(
        batches_expected, batches_produced)
            
            

In [ ]:
print("Loading data...")
data = load_data(glob.glob('data/*.out'))
train, val, test = split_data(*data)

print("Starting training...")
num_epochs = 10
for epoch in range(num_epochs):
    train_err = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(cross_join(train, max_alerts=None), 1000):
        train_err += train_fn(*batch)
        train_batches += 1
    
    #if (epoch+1) % (num_epochs/10) == 0:
    if True:
        val_err = 0
        val_acc = 0
        val_batches = 0
        for batch in iterate_minibatches(cross_join(val, max_alerts=None), 1000):
            err, acc = val_fn(*batch)
            val_err += err
            val_acc += acc
            val_batches += 1

        print("Epoch {} of {} took {:.3f}s".format(
            epoch + 1, num_epochs, time.time() - start_time))
        print("  training loss:\t\t{:.20f}".format(train_err / train_batches))
        print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
        print("  validation accuracy:\t\t{:.2f} %".format(
            val_acc / val_batches * 100))

test_err = 0
test_acc = 0
test_batches = 0
for batch in iterate_minibatches(cross_join(test, max_alerts=None), 1000):
    err, acc = val_fn(*batch)
    test_err += err
    test_acc += acc
    test_batches += 1
print("Final results:")
print("  test loss:\t\t\t{:.6f}".format(test_err / test_batches))
print("  test accuracy:\t\t{:.2f} %".format(
    test_acc / test_batches * 100))